# Session

Session digunakan untuk berkomunikasi langsung dengan database, misal kita membuat object dari sebuah model tetapi belum melakukan `session.commit` maka object tersebut tidak ada tersimpan di database. Pada flask-sqlalchemy, penggunaan session secara explisit hanya digunakan untuk perubahan atau penambahan object, sedangkan untuk query langsung pada model itu sendiri.


## Session.query

Digunakan untuk operasi 'select', pada flask-sqlalchemy penggunaannya dapat juga dilakukan dengan property `query` pada Model.

```python
product = db.session(Products).all()
# atau
product = Products.query.all()
```

## Session.add / Session.add_all

Untuk menambahkan object yang berubah atau object baru kedalam session.

```python
transaction = Transactions()
db.session.add(transaction)
db.session.flush()
```

## Session.flush

Melakukan `flush` ke database. maksud flush disini data yang sudah 'di flush' akan masuk ke database tetapi belum di simpan secara permanen sehingga harus dilakukan `commit` jika ingin menyimpan secara permanen

```python
transaction = Transactions()
db.session.add(transaction)
db.session.flush()
```

## Session.commit

Digunakan untuk menyimpan object secara permanent ke dalam database.

```python
transaction = Transactions()
db.session.add(transaction)
db.session.commit()
```


## Session.rollback

Ketika terjadi kesalahan maka session harus di `rollback` untuk membersihkan session

```python
transaction_product = TransactionProducts()
db.session.add(transaction_product)
db.session.commit()  # ini akan error karena pada transaction product ada data yang tidak diisi

db.session.rollback
```

## Contoh pada view transaction add

```python
@bp.route("/transaction/add", methods=["POST", "GET"])
def transaction_all():
    """Add new transaction"""
    if request.method == "GET":
        return render_template("transaction/form_add.html")

    products = request.form.getlist('products')
    products_qty = request.form.getlist('products_qty')

    transaction = Transactions()
    db.session.add(transaction)
    db.session.flush()

    for key, product in enumerate(products):
        transaction_product = TransactionProducts()
        transaction_product.transaction_id = transaction.id
        transaction_product.product_id = int(product)
        transaction_product.product_qty = int(products_qty[key])
        db.session.add(transaction_product)
        db.session.flush()

    db.session.commit()

    return redirect("/transaction")

```